In [79]:
import looker_sdk
from looker_sdk import models40 
ini_fp = r"C:\dev\Looker_Python\looker.ini"  
sdk = looker_sdk.init40(ini_fp)

In [80]:
def get_looker_models(ini_fp):
    # sdk = looker_sdk.init40(ini_fp)
    models = sdk.all_lookml_models(fields='name')
    model_names = [model.name for model in models]
    return model_names

def get_field_values(ini_fp, model_name, explore_name):

    # sdk = looker_sdk.init40(ini_fp)

    # API Call to pull in metadata about fields in a particular explore
    explore = sdk.lookml_model_explore(
        lookml_model_name=model_name,
        explore_name=explore_name,
        fields="id, name, description, fields",
    )

    my_fields = []

    # Iterate through the field definitions and pull in the description, sql, and other looker tags to include in data dictionary.
    if explore.fields and explore.fields.dimensions:
        for dimension in explore.fields.dimensions:
            dim_def = {
                "field_type": "Dimension",
                "view_name": dimension.view_label,
                "field_name": dimension.label_short,
                "type": dimension.type,
                "description": dimension.description,
                "sql": dimension.sql,
            }
            my_fields.append(dim_def)
    if explore.fields and explore.fields.measures:
        for measure in explore.fields.measures:
            mes_def = {
                "field_type": "Measure",
                "view_name": measure.view_label,
                "field_name": measure.label_short,
                "type": measure.type,
                "description": measure.description,
                "sql": measure.sql,
            }
            my_fields.append(mes_def)

    return my_fields

def get_explore_fields(ini_fp, model_name, explore_name):
    # sdk = looker_sdk.init40(ini_fp)

    # Retrieve the explore details
    explore = sdk.lookml_model_explore(model_name, explore_name)

    # Extracting the field names
    dimension_names = [dimension.name for dimension in explore.fields.dimensions]
    measure_names = [measure.name for measure in explore.fields.measures]

    return dimension_names + measure_names

def list_explores_for_each_model(ini_fp):
    # sdk = looker_sdk.init40(ini_fp)
    lookml_models = sdk.all_lookml_models(fields='name')
    
    model_explores = {}
    for model in lookml_models:
        explores = sdk.lookml_model(lookml_model_name=model.name, fields='explores')
        model_explores[model.name] = [explore.name for explore in explores.explores]

    return model_explores

def run_query_with_filters_json(ini_fp, model_name, explore_name, fields, filters):
    # sdk = looker_sdk.init40(ini_fp)
    query = {
        "model": model_name,
        "view": explore_name,
        "fields": fields,
    }

    result = sdk.run_inline_query("json", query)
    return result

def run_query_with_filters_csv(ini_fp, model_name, explore_name, fields, filters):
    # sdk = looker_sdk.init40(ini_fp)
    query = {
        "model": model_name,
        "view": explore_name,
        "fields": fields,
    }

    result = sdk.run_inline_query("csv", query)
    return result

model_name = "quantiphi_als_rrv"
explore_name_list = ['subject_data_rrv_2']
explore_name = explore_name_list[0]

print(f"\n\nAvailable Explores: {list_explores_for_each_model(ini_fp)}")





Available Explores: {'quantiphi_als_rrv': ['subject_tracker_data', 'voice_recording_data', 'subject_data_rrv_2'], 'extension-api-explorer': []}


In [81]:
fields = get_field_values(ini_fp, model_name, explore_name)
print(f"\n\nAvailable Fields for {model_name}: ")
for field in fields:
    print(field)





Available Fields for quantiphi_als_rrv: 
{'field_type': 'Dimension', 'view_name': 'ALSFRS Data', 'field_name': 'ALSFRS Entry Date', 'type': 'date_date', 'description': '', 'sql': None}
{'field_type': 'Dimension', 'view_name': 'ALSFRS Data', 'field_name': 'ALSFRS Entry Day of Week', 'type': 'date_day_of_week', 'description': '', 'sql': None}
{'field_type': 'Dimension', 'view_name': 'ALSFRS Data', 'field_name': 'ALSFRS Entry Month', 'type': 'date_month', 'description': '', 'sql': None}
{'field_type': 'Dimension', 'view_name': 'ALSFRS Data', 'field_name': 'ALSFRS Entry Quarter', 'type': 'date_quarter', 'description': '', 'sql': None}
{'field_type': 'Dimension', 'view_name': 'ALSFRS Data', 'field_name': 'ALSFRS Entry Raw', 'type': 'date_raw', 'description': '', 'sql': None}
{'field_type': 'Dimension', 'view_name': 'ALSFRS Data', 'field_name': 'ALSFRS Entry Time', 'type': 'date_time', 'description': '', 'sql': None}
{'field_type': 'Dimension', 'view_name': 'ALSFRS Data', 'field_name': 'AL

In [82]:
field_names = get_explore_fields(ini_fp, model_name, explore_name)
print("\n\nAvailable Field Names: ")
for name in field_names:
    print(name)






Available Field Names: 
als_frs_data.subject_form_entry_date
als_frs_data.subject_form_entry_day_of_week
als_frs_data.subject_form_entry_month
als_frs_data.subject_form_entry_quarter
als_frs_data.subject_form_entry_raw
als_frs_data.subject_form_entry_time
als_frs_data.subject_form_entry_week
als_frs_data.subject_form_entry_year
als_frs_data.days_since_onset
als_frs_data.days_since_sign_up
als_frs_data.dynamic_activity
als_frs_data.dynamic_activity_2
als_frs_data.frs_cutting_food
als_frs_data.frs_dressing_hygiene
als_frs_data.frs_dyspnea
als_frs_data.frs_handwriting
als_frs_data.frs_orthopnea
als_frs_data.frs_respiratory_insufficiency
als_frs_data.frs_salivation
als_frs_data.frs_speech
als_frs_data.frs_swallowing
als_frs_data.frs_turning_in_bed
als_frs_data.frs_walking
als_frs_data.frs_climbing_stairs
als_frs_data.subject_form_id
als_frs_data.subject_id
als_frs_data.total_score
als_frs_data.year_since_signup
als_frs_slope_data.partial_total
als_frs_slope_data.als_frs_slope_per_month
a

In [83]:
# # Example filter dict:
# filters = {
#     "field_name1": "filter_value1",
#     "field_name2": "filter_value2",
#     # Add as many field filters as needed
# }
filters = {}

# Example: query using the first ten fields returned
# fields.pop(11)
fields = field_names[0:20]
data = run_query_with_filters_csv(ini_fp, model_name, explore_name, fields, filters=filters)
print("\n\n")
print(data)

SDKError: 
    message:           Query execution failed:  - Name AS not found inside als_frs_data at [13:20]
    documentation_url: https://cloud.google.com/looker/docs/r/err/4.0/400/post/queries/run/:result_format
    error_doc_url:     https://static-a.cdn.looker.app/errorcodes/400.md
    error details:
    
    

In [ ]:
# Initialize the SDK (use init31() for the older v3.1 API)
# sdk = looker_sdk.init40()

# Get information about the authenticated user
my_user = sdk.me()

instance_url = sdk.get_setting(fields="host_url")
print(instance_url['host_url'])

# Output can be treated like a dictionary
print(my_user["first_name"])

# Or as a model instance (User in this case)
print(my_user.first_name)

https://alstdi.cloud.looker.com
Clifford
Clifford


In [ ]:
"""
This script will construct and output an object detailing content access metadata for each folder, 
providing an example of how to recursively traverse the folder tree. For each folder, the object will
contain a list of Groups and Users who have either been explicitly been granted view/edit access to 
the folder, or implicitly given via inheritence from a parent folder.

To include personal and/or the embed shared folders, modify the parameters below. 
"""

# import looker_sdk
import pprint as pp

# sdk = looker_sdk.init40()

# includes both folders under folders/users "People" folder and folders/embed_users "Embed Users"
INCLUDE_PERSONAL = False
# include folders under folders/embed_shared "Embed Groups" folder
INCLUDE_EMBED_SHARED = False


def get_folders():
    print('** gathering folder info **')
    all_folders = sdk.all_folders()  # get all folder metadata
    folder_access = {}

    for f in all_folders:
        personal = f.is_personal or f.is_personal_descendant
        if (personal or f.is_users_root or f.is_embed_users_root) and not INCLUDE_PERSONAL:
            continue
        if (not personal and f.is_embed) and not INCLUDE_EMBED_SHARED:
            continue
        folder_access[f.id] = {"folder_id": f.id,
                               "name": f.name,
                               "parent_id": f.parent_id,
                               "content_metadata_id": f.content_metadata_id,
                               "personal": personal,
                               "child_count": f.child_count}

    folder_access.pop('lookml')  # remove lookml dashboard folder
    return folder_access


def grab_folder_permissions(f: str, folders: object):
    print('checking folder: {}'.format(f))

    if 'access' in folders[f]:
        print('skipping folder {}: already have access details'.format(f))
    else:
        # groups / users can view / edit in folder (content_metadata_id)
        cmgu = sdk.all_content_metadata_accesses(
            content_metadata_id=folders[f]["content_metadata_id"], fields='group_id,user_id,permission_type')

        access = {"groups": [], "users": []}
        access["groups"] = [{"id": i.group_id, "permission_type": i.permission_type.value}
                            for i in cmgu if i.group_id is not None]
        access["users"] = [{"id": i.user_id, "permission_type": i.permission_type.value}
                           for i in cmgu if i.user_id is not None]
        folders[f]['access'] = access

    # folders / dash / looks in folder (content_metadata_id). use to check if folder 'inherits' content access
    cm = sdk.all_content_metadatas(
        parent_id=folders[f]['content_metadata_id'], fields='folder_id,inherits')

    # add access to all inheriting folders
    for c in cm:
        if c.folder_id is not None:
            try:
                if c.inherits:
                    print('{} inherits as parent {}'.format(c.folder_id, f))
                    folders[c.folder_id]['access'] = folders[f]['access']
                # recursively set child folders (depth first)
                grab_folder_permissions(c.folder_id, folders)
            except:
                # most likely the folder has been deleted
                print('skipping folder {}: no access data'.format(c.folder_id))

    return folders


def main():
    """Construct a dictionary of folder objects with access permissions:

    folder_id: {
        folder_id: string
        name: string
        parent_id: string
        content_metadata_id: string
        personal: bool based on is_personal or is_personal_descendant
        child_count: number of folders inside the folder
        access: {groups:[{id: int,permission_type: 'view' | 'edit'}], users:[{id: int,permission_type: 'view' | 'edit'}]}
    }

    admins have access to all content folders and are not included"""

    folders = get_folders()
    print('** gathering shared folders **')
    folder_access = grab_folder_permissions(
        '1', folders)  # start at folders/home
    if INCLUDE_PERSONAL or INCLUDE_EMBED_SHARED:
        print('** gathering personal and/or embed folders **')
        for f in folders:
            if 'access' not in folders[f]:
                folder_access = grab_folder_permissions(f, folders)

    pp.pprint(folder_access, sort_dicts=False)


main()

** gathering folder info **
** gathering shared folders **
checking folder: 1
8 inherits as parent 1
skipping folder 8: no access data
29 inherits as parent 1
checking folder: 29
skipping folder 29: already have access details
{'1': {'folder_id': '1',
       'name': 'Shared',
       'parent_id': None,
       'content_metadata_id': '1',
       'personal': False,
       'child_count': 1,
       'access': {'groups': [{'id': '1', 'permission_type': 'edit'}],
                  'users': []}},
 '29': {'folder_id': '29',
        'name': 'Main ALSTDI Dashboards',
        'parent_id': '1',
        'content_metadata_id': '125',
        'personal': False,
        'child_count': 0,
        'access': {'groups': [{'id': '1', 'permission_type': 'edit'}],
                   'users': []}},
 'lookml': {'folder_id': 'lookml',
            'name': 'LookML Dashboards',
            'parent_id': None,
            'content_metadata_id': None,
            'personal': False,
            'child_count': 0}}
